# Demo 2: Using Data from a SMART to Address Primary Aims about Embedded Adaptive Interventions

</br>
This material has been developed for [Getting SMART About Adaptive Interventions in Education](https://d3lab.isr.umich.edu/training/) led by [d3lab](https://d3lab.isr.umich.edu). 
    
Notebooks were developed by [Nicholas J. Seewald](https://nickseewald.com). 
SAS code originally written by Daniel Almirall, Inbal Nahum-Shani, and Susan A. Murphy.
The code was translated into R by Audrey Boruvka and Nicholas J. Seewald.

**Disclaimer:** This file provides example R code to analyze *simulated* data that was generated to mimic data arising from the ADHD SMART study (PI: William Pelham). An accompanying handout ("ADHD Handout.pdf") describes the variables in the data.

### Contents:
- [Part (a): Estimating the mean outcome under an embedded adaptive intervention](#part-a)
- [Part (b): Compare means from two embedded adaptive interventions](#part-b)
- [Part (c): Compare all four embedded AIs with one regression](#part-c)

<hr>

## Background
We'll be using the ADHD SMART (PI: Pelham) for this tutorial. A diagram of the study is below.

<img src="assets/adhd-smart-diagram.jpg" alt="ADHD SMART diagram" style="width: 50%;"/>

## Setup
As in the [Main Effects Demo](01_MainEffects_Demo.ipynb), we will begin by loading the data. We omit summaries here for brevity; see [01_MainEffects_Demo.ipynb](01_MainEffects_Demo.ipynb) for details on the data we will be using in this tutorial. The block of code below performs all necessary pre-processing steps from Module 3 related to loading and sorting the data, as well as centering covariates. **See [01_MainEffects_Demo.ipynb](01_MainEffects_Demo.ipynb) for details on this code; otherwise just run the cells and move to Part (a).**

In [1]:
library(geepack)
source('functions.R')

function 'estimate' loaded successfully.


In [2]:
adhd <- read.csv("assets/adhd-simulated-dataset.csv")

# R is case-sensitive! Avoid issues by changing variable names to all lowercase
names(adhd) <- tolower(names(adhd))

# Handle missing data: set "NaN" values to "NA" (R's missing data value)
adhd$o21[is.nan(adhd$o21)] <- NA
adhd$a2[is.nan(adhd$a2)] <- NA

# Sort data by ID
adhd <- adhd[order(adhd$id), ]

adhd$o11c <- with(adhd, o11 - mean(o11))
adhd$o12c <- with(adhd, o12 - mean(o12))
adhd$o13c <- with(adhd, o13 - mean(o13))
adhd$o14c <- with(adhd, o14 - mean(o14))
## o21 is measured among non-responders only
adhd$o22c <- with(adhd, o22 - mean(o22))

## center baseline variables using mean among non-responders
adhd$o11cnr <- adhd$o12cnr <- adhd$o13cnr <- adhd$o14cnr <- NA
adhd$o21cnr <- adhd$o22cnr <- NA
adhd$o11cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o11 - mean(o11))
adhd$o12cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o12 - mean(o12))
adhd$o13cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o13 - mean(o13))
adhd$o14cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o14 - mean(o14))
adhd$o21cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o21 - mean(o21))
adhd$o22cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o22 - mean(o22))

As a reminder, the `adhd` dataset contains the following variables. A more detailed summary is available in [this handout](https://gettingsmartworkshop-nseewald.notebooks.azure.com/j/files/ADHD_Data_Description_Handout.pdf).

| Variable Name | Variable Type | Description 
| -- | -- | :---
| `id` | Integer | Student ID number 
| `o11` | Baseline covariate; binary (0/1) | Oppositional defiant disorder diagnosis (1 = yes, 0 = no) 
| `o12` | Baseline covariate; continuous | ADHD severity at end of previous school year (0 - 3; higher values are fewer symptoms) 
| `o13` | Baseline covariate; binary (0/1) | Medication prior to first-stage intervention (1 = yes; 0 = no) 
| `o14` | Baseline covariate; binary (0/1) | Race (1 = white; 0 = non-white) 
| `a1` | First-stage intervention indicator (-1/1) | -1 = MED; -1 = BMOD 
| `r` | Response indicator (0/1) | 0 = Participant did not respond to first-stage intervention;  1 = participant responded
| `o21` | Intermediate outcome; continuous | Number of months until non-response (max of 8 months) 
| `o22` | Intermediate outcome; binary (0/1) | Adherence to first-stage intervention (1 = high adherence; 0 = otherwise)
| `a2` | Second-stage intervention indicator (-1/1) | -1 = AUGMENT; 1 = INTENSIFY
| `y` | Outcome (continuous) | Participant's school performance at end of school year (1-5; higher is better)

## <a name="part-a"></a>Part (a): Estimating the mean outcome under an embedded adaptive intervention
A common primary aim in a SMART is to compare the mean outcomes under two embedded adaptive interventions which recommend different first-stage intervention options. Before we can make that comparison, it's useful to explore how to estimate the mean outcome under *one* embedded adaptive intervention.

In the ADHD study, there are **four** embedded adaptive interventions:
1. MED  (A1 = -1) then AUGMENT   (A2 = -1) if non-response
2. BMOD (A1 = 1)  then AUGMENT   (A2 = -1) if non-response
3. MED  (A1 = -1) then INTENSIFY (A2 = 1)  if non-response
4. BMOD (A1 = 1)  then INTENSIFY (A2 = 1)  if non-response

<div style="display: flex; flex-wrap: wrap; padding: 0 4px;">
  <div class="column">
    <img src="assets/dtr1.jpg" style="margin-top: 8px; vertical-align: middle; width: 100%;">
    <img src="assets/dtr3.jpg" style="margin-top: 8px; vertical-align: middle; width: 100%;">
  </div>
  <div class="column">
    <img src="assets/dtr2.jpg" style="margin-top: 8px; vertical-align: middle; width: 100%;">
    <img src="assets/dtr4.jpg" style="margin-top: 8px; vertical-align: middle; width: 100%;">
  </div>
</div>

### Estimate the mean outcome under (MED, AUGMENT)
We'll focus on AI 1 (MED, AUGMENT) for now. The process is analogous for the other embedded AIs.

First, we'll create an indicator variable for whether or not an individual is consistent with (MED, AUGMENT). Call it `z1`. We define it as follows:
$$ Z_1 = \left\{
\begin{array}{rl}
    1 & \text{consistent with (MED, AUGMENT)} \\
    -1 & \text{otherwise}
\end{array}\right.. $$

In [3]:
# Everyone starts with z1 = -1 (MED); add 2 if a1 = 1 and a2 is either NA (if responder) or -1 (AUGMENT; if non-responder)
adhd$z1 <- -1 + 2 * with(adhd, a1 == -1 & a2 %in% c(NA, -1))

In order to account for imbalance (by design) in the numbers of responders and non-responders, we need to apply weights. Remember that the probability that a responder follows a given adaptive intervention is $1/2$, and the probability that a non-responder is consistent with an AI is $1/2 \times 1/2 = 1/4$. So, we'll weight responders by 2 and non-responders by 4.

We'll create a new variable in the data called `w`:

In [4]:
adhd$w <- with(adhd, 2 * r + 4 * (1-r))

We can now run a *weighted* regression. As above, we'll use `geeglm()`, this time specifying the `weights` argument in order to use the weights we've created. The model we'll use to estimate the mean outcome under AI 1 is
$$\mu(\mathbf{b}) := E[Y\mid Z_1] = b_0 + b_1 Z_1,$$
where $Z_1$ is the indicator we created above. `geeglm()` finds $\hat{\mathbf{b}}$ by solving the following estimating equations:
$$0 = E\left[W(A_1, R, A_2)\cdot D \cdot \left(Y - \mu(\mathbf{b})\right)\right].$$

In [5]:
model3 <- geeglm(y ~ z1, weights = w, id = id, data = adhd)
estimate(model3,
         rbind("Mean Y under AI #1 (MED, AUGMENT)" = c(1, 1)))

                                  Estimate 95% LCL 95% UCL     SE p-value
Mean Y under AI #1 (MED, AUGMENT)   2.8649  2.5305  3.1992 0.1706 < 1e-04

The estimated mean outcome had everyone in the SMART been treated consistent with (MED, AUGMENT) is 2.8649. The p-value is quite small, but remember this is for the test of $H_0: \mathbf{c}^\top \mathbf{b} = 0$ vs. $H_1: \mathbf{c}^\top \mathbf{b} \neq 0$. Here, this is a test for whether the mean outcome is zero, which is generally uninteresting. Our focus is on the estimate and confidence interval, here.

### Estimate the mean outcome under (BMOD, AUGMENT)
We can repeat the above procedure for the (BMOD, AUGMENT) adaptive intervention. We create a new indicator, $Z_2$, for consistency with this AI, then once again perform a weighted regression. ** sandwich**

In [6]:
# Everyone starts with a1 = 1 (BMOD); add 2 if a1 = 1 and a2 is either NA (if responder) or -1 (AUGMENT; if non-responder)
adhd$z2 <- -1 + 2 * with(adhd, a1 ==  1 & a2 %in% c(NA, -1))

# Run weighted regression
model4 <- geeglm(y ~ z2, weights = w, id = id, data = adhd)

estimate(model4,
         rbind("Mean Y under AI#2 (BMOD, AUGMENT)" = c(1, 1)))

                                  Estimate 95% LCL 95% UCL     SE p-value
Mean Y under AI#2 (BMOD, AUGMENT)   3.5067  3.1643  3.8490 0.1747 < 1e-04

## <a name="part-b"></a> Part (b): Compare means from two embedded adaptive interventions
The following code is used to estimate mean outcomes under AI 1, denoted (MED, AUGMENT), vs. AI 2, denoted (BMOD, AUGMENT). The goal is to do this simultaneously (i.e., with one regression). This also faciliates making inferences about the difference in the mean of these AIs.

The regression below represents an intuitive (yet less efficient) way to compare these adaptive interventions. We use only data from participants who are consistent with one of the two AIs we're comparing. We will do this with the `subset` argument to `geeglm()`. We will provide a logical statement (something that evaluates to either `TRUE` or `FALSE`) identifying the subset of observations we wish to use in the analysis. Here, we want to include only individuals who have $Z_1 = 1$ or $Z_2 = 1$, since these will be the individuals consistent with (MED, AUGMENT) and (BMOD, AUGMENT), respectively. In R code, this statement becomes `z1 == 1 | z2 == 1`. The double equals sign `==` is "logical equality" (as opposed to one equals sign, which is used to separate the argument names from their values), and the pipe `|` is ["logical OR"](https://en.wikipedia.org/wiki/Logical_disjunction). 

Using the `subset` argument to restrict ourselves to using only observations for which $Z_1=1$ or $Z_2 = 1$ has an interesting consequence: *only one indicator is needed in the regression*. Since, in our subset, if $Z_1 = 1$, then $Z_2$ must be $-1$, and vice versa. Therefore, we will fit the model
$$ E[Y\mid Z_1, \mathbf{SOMETHING ELSE}] = \gamma_0 + \gamma_1 Z_1.$$
Notice that $E[Y\mid Z_1=1] = \gamma_0 + \beta_1$ is the mean outcome under (MED, AUGMENT), and $E[Y\mid Z_1=-1]=\beta_0 - \beta_1$ is the mean outcome under (BMOD, AUGMENT). The difference in means is given by $2\beta_1$. 

In [7]:
## Conduct a regression analysis to compare mean outcomes under AI#1 vs AI#2
model5 <- geeglm(y ~ z1, weights = w, id = id, data = adhd,
                 # Only use individuals for whom z1 OR z2 is 1
                 subset = z1 == 1 | z2 == 1)

estimate(model5,
         rbind("Mean Y under AI#1 (MED, AUGMENT)"  = c(1,  1),
               "Mean Y under AI#2 (BMOD, AUGMENT)" = c(1, -1),
               "Difference AI#1 - AI#2"            = c(0,  2)))

                                  Estimate 95% LCL 95% UCL     SE p-value
Mean Y under AI#1 (MED, AUGMENT)    2.8649  2.5305  3.1992 0.1706  <1e-04
Mean Y under AI#2 (BMOD, AUGMENT)   3.5067  3.1643  3.8490 0.1747  <1e-04
Difference AI#1 - AI#2             -0.6418 -1.1203 -0.1633 0.2442  0.0086

The results suggest that AI 2, (BMOD, AUGMENT), causes a higher mean outcome than AI 1 (MED, AUGMENT), and the difference is statistically significant at the 5% level. Pay attention to the standard error of this contrast, though: $s.e.(2\hat{\beta_1})=0.2442$. 

An easy way to improve the statistical efficiency of this analysis (i.e., lower the standard error of $2\hat{\beta}_2$) is to adjust for prognostic baseline covariates. Here, we'll add $O_{12c}$ and $O_{14c}$ (prior medication and race, respectively)

In [8]:
## Conduct a regression analysis to compare mean outcomes under AI#1 vs AI#2
## adjusting for baseline covariates O12c and O14c;

model6 <- geeglm(y ~ z1 + o12c + o14c, weights = w, id = id, data = adhd,
                 # Only use individuals for whom z1 OR z2 is 1
                 subset = z1 == 1 | z2 == 1)
estimate(model6,
         rbind("Mean Y under AI#1 (MED, AUGMENT)"  = c(1,  1, 0, 0),
               "Mean Y under AI#2 (BMOD, AUGMENT)" = c(1, -1, 0, 0),
               "Difference AI#1 - AI#2"            = c(0,  2, 0, 0)))


                                  Estimate 95% LCL 95% UCL     SE p-value
Mean Y under AI#1 (MED, AUGMENT)    2.8842  2.5919  3.1765 0.1491  <1e-04
Mean Y under AI#2 (BMOD, AUGMENT)   3.3727  3.0542  3.6912 0.1625  <1e-04
Difference AI#1 - AI#2             -0.4884 -0.9282 -0.0487 0.2244  0.0295

Notice here that the standard error of the difference in means, $s.e.(2\hat{\beta}_1)$, has decreased here to $0.2244$: controlling for baseline covariates yields smaller standard errors and thus narrower confidence intervals. 

You may also notice that the parameter estimates have changed -- this is because we are now addressing a different scientific question than before. In `model5`, we are interested in finding the difference in mean outcomes between two embedded adaptive interventions; in `model6`, we are again interested in that difference in mean outcomes, but this time adjusted for a child's history of medication for ADHD, as well as the child's race.

## <a name="part-c"></a>Part (c): Compare all four embedded AIs with one regression

We have now seen strategies for estimating the mean outcome under a single embedded adaptive intervention and for comparing the mean outcomes of two embedded adaptive interventions. **cite INS psych methods**

Often, we are interested in performing a *single* regression to estimate and compare the mean outcomes of all embedded adaptive interventions. To do this, we will use a "weighted and replicated" regression approach.

Because responders in the ADHD SMART are consistent with more than one adaptive intervention, we need to trick R into using their data twice. We call this trick *"replication"*. We will create two observations for each responder, and, for each pair, assign one of them $A_2 = 1$ and the other $A_2 = -1$. 

Responders to med contribute to (bmod, add med) and (med, intensify)
Responders to bmod contribute to (med, add bmod) and (bmod, intensity)
So replicate responders, assigning original to a2 = -1 (augment)
and replicates to a2 = 1 (intensify)

We will create a new dataset, called `adhd.rep`, which contains the replicated data. The way we do this is to use `rbind()` (row bind) on two `data.frame` objects: the first is the original copy of `adhd`; the second is the subset of `adhd` which contains only responders. Each of those `data.frame`s also has a new variable, `obs`, which identifies the "original" observation and its "replicate". 

In [9]:
# Replicate the data
adhd.rep <- rbind(data.frame(adhd, obs = 1),
               data.frame(subset(adhd, r == 1), obs = 2))

Now, we'll assign second-stage treatment to each of the replicates. Depending on whether an individual is a responder, assign `a2 == -1` to responders with `obs == 1` and `a2 == 1` to `obs == 2` responders. Then sort the data.

In [10]:
adhd.rep$a2[adhd.rep$r == 1] <- c(-1, 1)[with(adhd.rep, obs[r == 1])]

## NOTE: geeglm needs data to be sorted by cluster id 
## (here a "cluster" is a single person in the SMART)
adhd.rep <- adhd.rep[order(adhd.rep$id), ]

We can check to see if the replication worked by cross-tabulating `a2` and `r`. In the first table, we look at the original data, `adhd`; in the second table, we examine `adhd.rep`.

In [11]:
## Cross-tabulate R and A2 in the original data
with(adhd, table(r, a2, useNA = "ifany"))

   a2
r   -1  1 <NA>
  0 49 50    0
  1  0  0   51

In [12]:
## For comparison, cross-tabulate R and A2 in the replicated data
with(adhd.rep, table(r, a2))

   a2
r   -1  1
  0 49 50
  1 51 51

In [13]:
## To see the replicated structure, print observations 45-65 in the new data
adhd.rep[45:65, ]

id o11 o12         o13 o14 a1 r o21 o22 a2 ... o14cnr     o13cnr    
32  32 0   -2.85948543 0   1    1 1 NA  0   -1 ...         NA         NA
321 32 0   -2.85948543 0   1    1 1 NA  0    1 ...         NA         NA
33  33 1   -1.19879945 1   1    1 0  6  1    1 ...  0.1919192  0.7070707
34  34 0    0.09602856 0   1    1 0  7  0    1 ...  0.1919192 -0.2929293
35  35 1    0.46216646 0   1    1 0  8  0   -1 ...  0.1919192 -0.2929293
36  36 1   -0.15586258 0   1   -1 0  1  0    1 ...  0.1919192 -0.2929293
37  37 0    0.87496507 0   1   -1 1 NA  1   -1 ...         NA         NA
371 37 0    0.87496507 0   1   -1 1 NA  1    1 ...         NA         NA
38  38 0    0.01974426 1   0   -1 0  5  1   -1 ... -0.8080808  0.7070707
39  39 1    1.52923952 1   1   -1 1 NA  1   -1 ...         NA         NA
391 39 1    1.52923952 1   1   -1 1 NA  1    1 ...         NA         NA
40  40 1    0.42969104 0   0    1 0  4  1   -1 ... -0.8080808 -0.2929293
41  41 0    0.38763663 1   1   -1 1 NA  0   -1 ...         NA         NA
411 41 0    0.38763663 1   1   -1 1 NA  0    1 ...         NA         NA
42  42 0    0.38898363 0   1    1 0  3  0   -1 ...  0.1919192 -0.2929293
43  43 0    0.15206684 0   0    1 1 NA  1   -1 ...         NA         NA
431 43 0    0.15206684 0   0    1 1 NA  1    1 ...         NA         NA
44  44 0    1.26222594 0   1   -1 0  2  0   -1 ...  0.1919192 -0.2929293
45  45 1    1.07428564 1   1   -1 1 NA  0   -1 ...         NA         NA
451 45 1    1.07428564 1   1   -1 1 NA  0    1 ...         NA         NA
46  46 0   -1.42125894 0   0   -1 0  1  0   -1 ... -0.8080808 -0.2929293
    o12cnr      o11cnr     o22cnr     o21cnr     z1 w z2 obs
32           NA         NA         NA         NA -1 2  1 1  
321          NA         NA         NA         NA -1 2  1 2  
33  -1.05412743  0.6767677  0.5252525  1.4141414 -1 4 -1 1  
34   0.24070059 -0.3232323 -0.4747475  2.4141414 -1 4 -1 1  
35   0.60683848  0.6767677 -0.4747475  3.4141414 -1 4  1 1  
36  -0.01119055  0.6767677 -0.4747475 -3.5858586 -1 4 -1 1  
37           NA         NA         NA         NA  1 2 -1 1  
371          NA         NA         NA         NA  1 2 -1 2  
38   0.16441628 -0.3232323  0.5252525  0.4141414  1 4 -1 1  
39           NA         NA         NA         NA  1 2 -1 1  
391          NA         NA         NA         NA  1 2 -1 2  
40   0.57436307  0.6767677  0.5252525 -0.5858586 -1 4  1 1  
41           NA         NA         NA         NA  1 2 -1 1  
411          NA         NA         NA         NA  1 2 -1 2  
42   0.53365566 -0.3232323 -0.4747475 -1.5858586 -1 4  1 1  
43           NA         NA         NA         NA -1 2  1 1  
431          NA         NA         NA         NA -1 2  1 2  
44   1.40689797 -0.3232323 -0.4747475 -2.5858586  1 4 -1 1  
45           NA         NA         NA         NA  1 2 -1 1  
451          NA         NA         NA         NA  1 2 -1 2  
46  -1.27658692 -0.3232323 -0.4747475 -3.5858586  1 4 -1 1

The model we'll fit is as follows:
$$E[Y\mid A_1, A_2] = \beta_0 + \beta_1 A_1 + \beta_2 A_2 + \beta_3 A_1 A_2.$$
Notice that the combination of $A_1$ and $A_2$ uniquely identifies an embedded adaptive intervention, and so we can estimate means of all embededed AIs using this model:

| Adaptive Intervention | &nbsp;&nbsp; Mean $Y$ under AI &nbsp;&nbsp;&nbsp;&nbsp; |
|-----------------------|:--------------------------------------------:|
| (MED, AUGMENT) = (-1, -1) | $\beta_0 - \beta_1 - \beta_2 + \beta_3$ |
| (BMOD, AUGMENT) = (1, -1) | $\beta_0 + \beta_1 - \beta_2 - \beta_3$ |
| (MED, INTENSIFY) = (-1, 1) | $\beta_0 - \beta_1 + \beta_2 - \beta_3$ |
| (BMOD, INTENSIFY) = (1, 1) | $\beta_0 + \beta_1 + \beta_2 + \beta_3$ | 

As before, we can identify linear combinations of the regression parameters which represent differences in mean outcomes between two embedded adaptive interventions. For example, the contrast we considered in [Part (a)](#part-a) (`model5`) between (MED, AUGMENT) and (BMOD, AUGMENT) is now written as $E[Y\mid A_1 = -1, A_2 = -1] - E[Y\mid A_1 = 1, A_2 = -1] = -2\beta_1 + 2\beta_3$. 

We input the model into R as `y ~ a1 * a2`. The `*` character is a shortcut for main effects of `a1` and `a2`, as well as their interaction (programmed in R as `a1:a2`). 

In [14]:
## Conduct a regression analysis to compare mean outcomes under AI#1 vs AI#2
## x:z = interaction term between x and z
## x*z is shorthand for: x + z + x:z
model7 <- geeglm(y ~ a1*a2, weights = w, id = id, data = adhd.rep)

estimate(model7,
         rbind(
           # These statements get the mean under each AI
           "Mean Y: AI#1 (MED, AUGMENT)"    = c(1, -1, -1,  1),
           "Mean Y: AI#2 (BMOD, AUGMENT)"   = c(1,  1, -1, -1),
           "Mean Y: AI#3 (MED, INTENSIFY)"  = c(1, -1,  1, -1),
           "Mean Y: AI#4 (BMOD, INTENSIFY)" = c(1,  1,  1,  1),
           # These statements are to get all pairwise comparisons
           "Diff: AI#1 - AI#2"              = c(0, -2,  0,  2),
           "Diff: AI#1 - AI#3"              = c(0,  0, -2,  2),
           "Diff: AI#1 - AI#4"              = c(0, -2, -2,  0),
           "Diff: AI#2 - AI#3"              = c(0,  2, -2,  0),
           "Diff: AI#2 - AI#4"              = c(0,  0, -2, -2),
           "Diff: AI#3 - AI#4"              = c(0, -2,  0, -2)))

                               Estimate 95% LCL 95% UCL     SE p-value
Mean Y: AI#1 (MED, AUGMENT)      2.8649  2.5305  3.1992 0.1706  <1e-04
Mean Y: AI#2 (BMOD, AUGMENT)     3.5067  3.1643  3.8490 0.1747  <1e-04
Mean Y: AI#3 (MED, INTENSIFY)    2.7895  2.4644  3.1145 0.1658  <1e-04
Mean Y: AI#4 (BMOD, INTENSIFY)   2.6533  2.2515  3.0552 0.2050  <1e-04
Diff: AI#1 - AI#2               -0.6418 -1.1203 -0.1633 0.2442  0.0086
Diff: AI#1 - AI#3                0.0754 -0.3106  0.4614 0.1969  0.7018
Diff: AI#1 - AI#4                0.2115 -0.3112  0.7343 0.2667  0.4277
Diff: AI#2 - AI#3                0.7172  0.2451  1.1893 0.2409  0.0029
Diff: AI#2 - AI#4                0.8533  0.3733  1.3334 0.2449  0.0005
Diff: AI#3 - AI#4                0.1361 -0.3807  0.6530 0.2637  0.6057

Notice that the estimated difference in mean outcomes between (MED, AUGMENT) and (BMOD, AUGMENT), $-2\hat{\beta}_1 + 2\hat{\beta}_3$, is identical to the result from `model5` in [Part (a)](#part-a) (including the standard error). The weighted-and-replicated regression allows us to achieve the same results as individual regressions for particular contrasts, but with the benefit of allowing us to examine all such contrasts simultaneously. 

As before, we can adjust for prognostic baseline covariates in order to improve the efficiency of the estimates. We'll again add $O_{12c}$ and $O_{14c}$ to the model. *Note that we are careful to not adjust for post-baseline or time-varying covariates or intermediate outcomes here -- doing so will lead to causal bias.*

In [15]:
model8 <- geeglm(y ~ a1 + a2 + o12c + o14c + a1:a2, weights = w, id = id,
                 data = adhd.rep)

estimate(model8,
         rbind("Mean Y: AI#1 (MED, AUGMENT)"    = c(1, -1, -1, 0, 0,  1),
               "Mean Y: AI#2 (BMOD, AUGMENT)"   = c(1,  1, -1, 0, 0, -1),
               "Mean Y: AI#3 (MED, INTENSIFY)"  = c(1, -1,  1, 0, 0, -1),
               "Mean Y: AI#4 (BMOD, INTENSIFY)" = c(1,  1,  1, 0, 0,  1),
               "Diff: AI#1 - AI#2"              = c(0, -2,  0, 0, 0,  2),
               "Diff: AI#1 - AI#3"              = c(0,  0, -2, 0, 0,  2),
               "Diff: AI#1 - AI#4"              = c(0, -2, -2, 0, 0,  0),
               "Diff: AI#2 - AI#3"              = c(0,  2, -2, 0, 0,  0),
               "Diff: AI#2 - AI#4"              = c(0,  0, -2, 0, 0, -2),
               "Diff: AI#3 - AI#4"              = c(0, -2,  0, 0, 0, -2)))

                               Estimate 95% LCL 95% UCL     SE p-value
Mean Y: AI#1 (MED, AUGMENT)      2.8801  2.5869  3.1733 0.1496  <1e-04
Mean Y: AI#2 (BMOD, AUGMENT)     3.3854  3.0689  3.7018 0.1614  <1e-04
Mean Y: AI#3 (MED, INTENSIFY)    2.8149  2.5163  3.1135 0.1524  <1e-04
Mean Y: AI#4 (BMOD, INTENSIFY)   2.7338  2.3596  3.1081 0.1909  <1e-04
Diff: AI#1 - AI#2               -0.5053 -0.9401 -0.0704 0.2219  0.0228
Diff: AI#1 - AI#3                0.0652 -0.2811  0.4115 0.1767  0.7120
Diff: AI#1 - AI#4                0.1463 -0.3271  0.6197 0.2415  0.5447
Diff: AI#2 - AI#3                0.5705  0.1321  1.0088 0.2236  0.0107
Diff: AI#2 - AI#4                0.6516  0.2100  1.0931 0.2253  0.0038
Diff: AI#3 - AI#4                0.0811 -0.3967  0.5588 0.2437  0.7394

Notice, now that we've adjusted for basline, we again see improved efficiency in our estimates relative to `model7` above. However, even more interestingly, we also see improved efficiency relative to `model6`, in which we compared two adaptive interventions while adjusting for these same baseline covariates. The point estimates remain about the same (though, again, when we adjust for covariates we are addressing a slightly different scientific question relative to when we do not).